# 从网页中截取文章内容

## 步骤1：参数设置

In [41]:
url = 'http://www.dataguru.cn/article-10828-1.html'
print(url)

http://www.dataguru.cn/article-10828-1.html


## 步骤2：查找相应的截取关键字

In [42]:
if 'blog.jobbole.com' in url:  # 伯乐在线
    title_key = '.entry-header'
    content_key = '.entry'
    print('伯乐在线 http://blog.jobbole.com')
elif 'blog.csdn.net' in url:  # csdn
    title_key = '.link_title'
    content_key = '#article_content'
    print('csdn http://blog.csdn.net')
elif 'www.codingpy.com' in url:  # 编程派网址
    title_key = '.header h1'
    content_key = '.article-content'
    print('编程派网址 http://www.codingpy.com')
elif 'www.dataguru.cn' in url:  # 炼数成金
    title_key = '.ph'
    content_key = '#article_content'
    print('炼数成金 http://www.dataguru.cn')
else:
    title_key = '#keraspython'
    content_key = '.section'
    print('其它')

炼数成金 http://www.dataguru.cn


## 步骤3：发出网页请求，接收响应

In [43]:
from urllib import request
req = request.Request(url)
res = request.urlopen(req)
html = res.read().decode('GBK')
# html = res.read().decode('gb2312')
# html = res.read().decode('utf-8')
print(html)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=gbk" />
<title>Tensorflow代码解析（四） - 深度学习-炼数成金-Dataguru专业数据分析社区</title>
   
<meta name="description" content="TF把神经网络模型表达成一张拓扑结构的Graph，Graph中的一个节点表示一种计算算子。Graph从输入到输出的Tensor数据流动完成了一个运算过程，这是对类似概率图、神经网络等连接式算法很好的表达，同时也是对Tensor +  ... " />
    <meta name="viewport" content="width=1100">
<meta name="generator" content="Discuz! X3.2" />
<meta name="author" content="Discuz! Team and Comsenz UI Team" />
<meta name="copyright" content="2001-2013 Comsenz Inc." />
<meta name="MSSmartTagsPreventParsing" content="True" />
<meta http-equiv="MSThemeCompatible" content="Yes" />
<base href="http://www.dataguru.cn/" /><link rel="stylesheet" type="text/css" href="data/cache/style_4_common.css?Wz8" /><link rel="stylesheet" type="text/css" href="data/cache/style_4_portal_view.css

## 步骤4：从网页中提取文章标题和内容

In [44]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.select(title_key)[0].text.strip()  # 文章标题
print('文章标题：', title)
content = soup.select(content_key)[0]  # 文章内容

文章标题： Tensorflow代码解析（四）


## 步骤5：下载文章中的图片

### 5.1 哈希码生成函数，用于给图片重新命名

In [45]:
import hashlib
def md5(name):
    """ 将字符串转成哈希码 """
    if not isinstance(name, str):
        name = str(name)
    md5 = hashlib.md5()
    md5.update(name.encode('utf-8'))
    return md5.hexdigest()

### 5.2 下载图片，并修改文章图片的超链接

In [46]:
import re
import os
content = str(content)
pattern = '<img .*?src=\"(.*?)\"'
re_image = re.compile(pattern)
for image_link in re_image.findall(content):
    if not os.path.exists('images'):
        os.mkdir('images')
    filename = 'images/' + md5(image_link) + os.path.splitext(image_link)[-1]
    try:
        request.urlretrieve(image_link, filename)
        print('下载完成', filename)
    except Exception as e:
        print('图片出错', e)
    else:
        content = content.replace(image_link, filename)
print('已完成--------')

图片出错 unknown url type: '/static/image/common/tag1.png'
图片出错 unknown url type: '/static/image/common/tag1.png'
图片出错 unknown url type: '/static/image/common/tag1.png'
图片出错 unknown url type: '/static/image/common/tag1.png'
图片出错 unknown url type: '/static/image/common/tag1.png'
图片出错 unknown url type: '/static/image/common/tag1.png'
下载完成 images/f27ec5c881f67eaed72ae53174689838.jpg
下载完成 images/e8c76482ceaa0db30ed3b252e9d1ce7a.jpg
下载完成 images/fee676903146b7847d75db60ac2707f3.jpg
下载完成 images/7cf7b78a6186e1c4cfb20cccf6bbe9e3.jpg
下载完成 images/7f5a645699ad9c2106cb3ff44b9ad921.jpg
下载完成 images/07585d6a6b437be9625c62119c6b30d7.jpg
下载完成 images/3022b2e36bc8066419714b26c427fed7.png
下载完成 images/4d8bfd918dc573c9c1d66385bcb10cf3.png
下载完成 images/aea14b4ef0dae7617b7cc8798b9edb29.jpg
下载完成 images/cc7824491e7a3ab18cd0bc136fdf6a29.png
下载完成 images/5c1f60c301aa5ed354fd47a7e8ebbc52.png
下载完成 images/a6fe99708f4158078338abe1ff2749a1.png
下载完成 images/4d847c742ec662cc1dd765a31d72bf84.jpg
下载完成 images/c6cd551fffb8ce0b46030

## 步骤6：将截取的文章标题和内容重新组合成新的网页文件

In [47]:
html_template = """<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="{origin}">原文链接</a></p>
<p><center><h1>{title}</h1></center></p>
    {content}
</body></html>"""
html = html_template.format(origin=url, title=title, content=content)
print(html)

<!DOCTYPE html>
<html><head><meta charset="UTF-8">
</head><body>
<p><a href="http://www.dataguru.cn/article-10828-1.html">原文链接</a></p>
<p><center><h1>Tensorflow代码解析（四）</h1></center></p>
    <td id="article_content">
<p style="margin-bottom:10px">
<span class="newtag"><img src="/static/image/common/tag1.png"/><a href="http://www.dataguru.cn/tags/网络" target="_blank">网络</a></span>
<span class="newtag"><img src="/static/image/common/tag1.png"/><a href="http://www.dataguru.cn/tags/算法" target="_blank">算法</a></span>
<span class="newtag"><img src="/static/image/common/tag1.png"/><a href="http://www.dataguru.cn/tags/模型" target="_blank">模型</a></span>
<span class="newtag"><img src="/static/image/common/tag1.png"/><a href="http://www.dataguru.cn/tags/分布式" target="_blank">分布式</a></span>
<span class="newtag"><img src="/static/image/common/tag1.png"/><a href="http://www.dataguru.cn/tags/GPU" target="_blank">GPU</a></span>
<span class="newtag"><img src="/static/image/common/tag1.png"/><a href="http://

## 步骤7：将文件写入磁盘

In [48]:
filename = title + ".html"
with open(filename, "w") as f:
    f.write(html.replace(u'\xa0', u' ').replace(u'\U0001f60a', u' '))  # 在windows中出错，所以这里进行了字符串替换

## (完)